## Overview
This notebook gives a look at the "closure" of the C cycle for `dvmdostem`.

The idea is to run the model for a short period of time and then check the model-outputs to make sure that the ecosystem-wide C stock's change from time step to time-step ("delta C") is the same as the net ecosystem C flux.

## Code
First, import a bunch of libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import textwrap

In [ ]:
%load_ext autoreload
%autoreload 2

Next, setup some functions that can compile the numbers we are interested in from a json object. The json object will come from loading the json files that `dvmdostem` produces when running with the `--cal-mode` flag.

In [ ]:
from . import diagnostics

In [ ]:
#print diagnostics.compile_table_by_year('C_veg_balance', fileslice='-15::')

In [ ]:
tl = sorted([
         'N_soil_balance',
#         'N_veg_balance',
         'C_soil_balance',
#         'C_veg_balance',
#         'C_veg_vascular_balance',
#         'C_veg_nonvascular_balance',
#         'report_soil_C'
])

diagnostics.run_tests(tl, p2c=True, fileslice=':100:')

In [ ]:
diagnostics.run_tests(['N_veg_balance'],p2c=True)

In [ ]:
#!cat ../config/calibration_directives.txt
#!cd .. && ./dvmdostem -l fatal -c -p 10

diagnostics.run_tests(['C_veg_balance',], p2c=True)

In [ ]:
diagnostics.plot_tests([
#'N_soil_balance',
#        'N_veg_balance',
      #  'C_soil_balance',
      #  'C_veg_balance',
#        'C_veg_vascular_balance',
#        'C_veg_nonvascular_balance',
        ])

Now we have all the tools in place to perform a model run and look at a quick summary of the C cycle.

For this experiment, we want to compare how things look with different modules on or off. We can control the module state using the "calibration_directives.txt" file. So our process will look like this:

- Create calibration_directives file with module settings.
- Run the model (controlled by the directives file.
- Anylyze the json files to get the C cycle report.
- Compare the reports from model runs with different module on/off states.

First we make a convenience wrapper for writing a calibration directives file, then run 3 different tests.

In [ ]:
def write_cal_directives(s):
    with open ("../config/calibration_directives.txt", 'w') as f:
        f.write(s)

In [ ]:
write_cal_directives(textwrap.dedent('''\
    {
      "calibration_autorun_settings": {
        "pwup": false,
        "quitat": 15
      }
    }
'''))
!cd ../ && ./dvmdostem --log-level fatal --cal-mode --pre-run-yrs 10
run_tests(['C_veg_balance', 'C_soil_balance','report_soil_C'], w2f="nfeedON-dslON/C_report.txt")
#!cd ../ && ./scripts/bulk-plot.sh --numpfts 7 --parallel --outdir "scripts/all-modules-on" --tag test0

In [ ]:
write_cal_directives(textwrap.dedent('''\
    {
      "calibration_autorun_settings": {
        "0": ["nfeed off", "dsl off"],
        "pwup": false,
        "quitat": 15
      }
    }
'''))
!cat ../config/calibration_directives.txt
!cd ../ && ./dvmdostem --log-level fatal --cal-mode --pre-run-yrs 10
run_tests(['C_veg_balance', 'C_soil_balance','report_soil_C'], w2f="nfeedOFF-dslOFF/C_report.txt")
#!cd ../ && ./scripts/bulk-plot.sh --numpfts 7 --parallel --outdir scripts/nfeedoff-dsloff --tag test1

In [ ]:
write_cal_directives(textwrap.dedent('''\
    {
      "calibration_autorun_settings": {
        "0": ["nfeed on", "dsl off"],
        "pwup": false,
        "quitat": 15
      }
    }
'''))
!cat ../config/calibration_directives.txt
!cd ../ && ./dvmdostem --log-level fatal --cal-mode --pre-run-yrs 10
run_tests(['C_veg_balance', 'C_soil_balance','report_soil_C'], w2f="nfeedON-dslOFF/C_report.txt")
#!cd ../ && ./scripts/bulk-plot.sh --numpfts 7 --parallel --outdir scripts/nfeedON-dsloff --tag test1

In [ ]:
write_cal_directives(textwrap.dedent('''\
    {
      "calibration_autorun_settings": {
        "0": ["nfeed off", "dsl on"],
        "pwup": false,
        "quitat": 15
      }
    }
'''))
!cd ../ && ./dvmdostem --log-level fatal --cal-mode --pre-run-yrs 10
run_tests(['C_veg_balance', 'C_soil_balance','report_soil_C'], w2f="nfeedOFF-dslON/report.txt")
#!cd ../ && ./scripts/bulk-plot.sh --numpfts 7 --parallel --outdir scripts/nfeedoff-dslON --tag test1